In [4]:
import pandas as pd

In [5]:
df=pd.read_csv("C:\\Users\\Home\\Desktop\\fake-news\\train.csv")

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [9]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
df=df.fillna('')

In [11]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [12]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [13]:
df=df.drop(['id', 'title', 'author'], axis=1)

In [14]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
import nltk

In [16]:
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.stem.porter import PorterStemmer

In [19]:
import re

In [20]:
port_stem=PorterStemmer()

In [21]:
port_stem.stem("Salam Habibi , waLla *&#^@")

'salam habibi , walla *&#^@'

In [22]:
def stemming(content):
  con=re.sub('[^a-zA-Z]',' ',content)
  con=con.lower()
  con=con.split()
  con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
  con=' '.join(con)
  return con


In [23]:
stemming('Salam Habibi , waLla *&#^@')

'salam habibi walla'

In [24]:
df['text']=df['text'].apply(stemming)

In [25]:
x=df['text']

In [26]:
y=df['label']

In [27]:
print("Text Columns: ",x.shape,", Label Columns: ",y.shape)

Text Columns:  (20800,) , Label Columns:  (20800,)


In [28]:
import sklearn
from sklearn.model_selection import train_test_split

In [29]:
x_train, x_test, y_train , y_test=train_test_split(x,y,test_size=0.25)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
vect=TfidfVectorizer()

In [32]:
x_train=vect.fit_transform(x_train)

In [33]:
x_test=vect.transform(x_test)

In [34]:
print("x_train ",x_train.shape,"x_test: ",x_test.shape)

x_train  (15600, 95726) x_test:  (5200, 95726)


In [35]:
from sklearn.tree import DecisionTreeClassifier

In [36]:
model=DecisionTreeClassifier()

In [37]:
model.fit(x_train,y_train)

DecisionTreeClassifier()

In [38]:
predictions=model.predict(x_test)

In [39]:
predictions

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [40]:
model.score(x_test,y_test)

0.8861538461538462

In [41]:
import pickle

In [43]:
pickle.dump(vect,open('D:\\university_files\\Semester_4\\Jupyter_Tasks\\pai_ccp\\vector.pkl','wb'))

In [44]:
pickle.dump(model,open('D:\\university_files\\Semester_4\\Jupyter_Tasks\\pai_ccp\\model.pkl','wb'))

In [45]:
vector_form=pickle.load(open('D:\\university_files\\Semester_4\\Jupyter_Tasks\\pai_ccp\\vector.pkl','rb'))

In [46]:
load_model=pickle.load(open('D:\\university_files\\Semester_4\\Jupyter_Tasks\\pai_ccp\\model.pkl','rb'))

In [47]:
def fake_news(news):
  news=stemming(news)
  input_data=[news]
  vector_form1=vector_form.transform(input_data)
  prediction=load_model.predict(vector_form1)
  return prediction

In [48]:
val=fake_news("""In these trying times, Jackie Mason is the Voice of Reason. [In this weekâ€™s exclusive clip for Breitbart News, Jackie discusses the looming threat of North Korea, and explains how President Donald Trump could win the support of the Hollywood left if the U. S. needs to strike first.  â€œIf he decides to bomb them, the whole country will be behind him, because everybody will realize he had no choice and that was the only thing to do,â€ Jackie says. â€œExcept the Hollywood left. Theyâ€™ll get nauseous. â€ â€œ[Trump] could win the left over, theyâ€™ll fall in love with him in a minute. If he bombed them for a better reason,â€ Jackie explains. â€œLike if they have no transgender toilets. â€ Jackie also says itâ€™s no surprise that Hollywood celebrities didnâ€™t support Trumpâ€™s strike on a Syrian airfield this month. â€œThey were infuriated,â€ he says. â€œBecause it might only save lives. That doesnâ€™t mean anything to them. If it only saved the environment, or climate change! Theyâ€™d be the happiest people in the world. â€ Still, Jackie says heâ€™s got nothing against Hollywood celebs. Theyâ€™ve got a tough life in this country. Watch Jackieâ€™s latest clip above.   Follow Daniel Nussbaum on Twitter: @dznussbaum""")
if val==[0]:
  print('Reliable')
else:
  print('Unreliable')

Reliable
